# Cycle-Consistent Adversarial Networks

### Theory:

The generator process:

<img src="assets/generator.png" style="width: 600px;"/>

The discriminator process:

<img src="assets/discriminator.png" style="width: 600px;"/>
 

### Result Improvement by
* Data augmentation
* Resize convolution[4]
* Instance normalization[5]


#### data augmentation:
<img src="assets/data_augmentation.png" style="width: 600px;"/>

#### Instance normalization and comparision by original paper https://arxiv.org/abs/1607.08022:
<img src="assets/instance_norm.png" style="width: 600px;"/>


#### Resize convolution (Remove Checkerboard Artifacts):

<img src="assets/compare1.png" style="width: 600px;"/>

<img src="assets/compare2.png" style="width: 600px;"/>


In [1]:
import os
import pprint
import numpy as np
import tensorflow as tf
import tensorlayer as tl
import argparse
from collections import namedtuple
from tensorlayer.layers import *
from tensorlayer.prepro import *
from random import shuffle
from model_deconv import *

In [2]:
epoch =  1   #200  #Epoch to train [100]
learning_rate = 0.0002  #Learning rate of for adam [0.0002]
beta1 = 0.5  #Momentum term of adam [0.5]
weight_decay = 1e-5  #Weight decay for l2 loss
pool_size = 50  #size of image buffer that stores previously generated images, default: 50
train_size = np.inf  #The size of train images [np.inf]
batch_size = 1  #The number of batch images [1] if we use InstanceNormLayer !
image_size = 256  #The size of image to use (will be center cropped) [256]
gf_dim = 32  #Size of generator filters in first layer
df_dim = 64  #Size of discriminator filters in first layer
class_embedding_size = 5  #Size of class embedding
output_size = 256  #The size of the output images to produce [64]
sample_size = 64  #The number of sample images [64]
c_dim = 3  #Dimension of image color. [3]
sample_step = 500  #The interval of generating sample. [500]
save_step = 200  #The interval of saveing checkpoints. [200]

dataset_dir = "sunflower2daisy"  #The name of dataset [horse2zebra, apple2orange, sunflower2daisy and etc]
checkpoint_dir = "checkpoint"  #Directory name to save the checkpoints [checkpoint]
sample_dir = "samples"  #Directory name to save the image samples [samples]
direction = "forward"  #The direction of generator [forward, backward]
test_dir = "./test"  #The direction of test
is_train = True   #True for training, False for testing [False]
is_crop = False  #True for training, False for testing [False]

In [3]:
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

In [4]:
sess = tf.Session()

##  Train CycleGAN

In [5]:
ni = int(np.sqrt(batch_size))
h, w = 256, 256
lamda = 10
# num_fake = 0

In [6]:
## data augmentation
def prepro(x):
    x = tl.prepro.flip_axis(x, axis=1, is_random=True)
    x = tl.prepro.rotation(x, rg=16, is_random=True, fill_mode='nearest')
    x = tl.prepro.imresize(x, size=[int(h * 1.2), int(w * 1.2)], interp='bicubic', mode=None)
    x = tl.prepro.crop(x, wrg=h, hrg=w, is_random=True)
    x = x / (255. / 2.)
    x = x - 1.
    return x

In [7]:
## data rescale 
def rescale(x):
    x = x / (255. / 2.)
    x = x - 1.
    return x

In [8]:
real_A = tf.placeholder(tf.float32, [None, image_size, image_size, c_dim], name='real_A')
real_B = tf.placeholder(tf.float32, [None, image_size, image_size, c_dim], name='real_B')
fake_A_pool = tf.placeholder(tf.float32, [batch_size, image_size, image_size, c_dim], name='fake_A')
fake_B_pool = tf.placeholder(tf.float32, [batch_size, image_size, image_size, c_dim], name='fake_B')

In [9]:
gen_B, gen_B_out = cyclegan_generator_resnet(real_A, 9, is_train=True, reuse=False, name='gen_A2B')
gen_A, gen_A_out = cyclegan_generator_resnet(real_B, 9, is_train=True, reuse=False, name='gen_B2A')
cyc_B, cyc_B_out = cyclegan_generator_resnet(gen_A_out, 9, is_train=True, reuse=True, name='gen_A2B')
cyc_A, cyc_A_out = cyclegan_generator_resnet(gen_B_out, 9, is_train=True, reuse=True, name='gen_B2A')

d_real_A, d_real_A_logits = cyclegan_discriminator_patch(real_A, is_train=True, reuse=False, name='dis_A')  
# dx
d_real_B, d_real_B_logits = cyclegan_discriminator_patch(real_B, is_train=True, reuse=False, name='dis_B')  
# dy
d_fake_A, d_fake_A_logits = cyclegan_discriminator_patch(gen_A_out, is_train=True, reuse=True, name='dis_A')  
# d_fy
d_fake_B, d_fake_B_logits = cyclegan_discriminator_patch(gen_B_out, is_train=True, reuse=True, name='dis_B')  
# d_gx

d_A_pool, d_A_pool_logits = cyclegan_discriminator_patch(fake_A_pool, is_train=True, reuse=True, name='dis_A')  
# d_fakex
d_B_pool, d_B_pool_logits = cyclegan_discriminator_patch(fake_B_pool, is_train=True, reuse=True, name='dis_B')  
# d_fakey

  [TL] InputLayer  gen_A2B/in: (?, 256, 256, 3)
  [TL] Conv2dLayer gen_A2B/c7s1-32: shape:[7, 7, 3, 32] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B/ins1: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B/d64: shape:[3, 3, 32, 64] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B/ins2: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B/d128: shape:[3, 3, 64, 128] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B/ins3: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B/res/c1/0: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B/res/ins/0: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B/res/c2/0: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B/res/ins2/0: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_A2B/b_residual_add/0: size:(?, 64, 64, 128) fn:add
  [TL] Conv2dLayer gen_A2B/res/c1/

  [TL] Conv2dLayer gen_B2A/res/c2/6: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins2/6: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_B2A/b_residual_add/6: size:(?, 64, 64, 128) fn:add
  [TL] Conv2dLayer gen_B2A/res/c1/7: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins/7: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_B2A/res/c2/7: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins2/7: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_B2A/b_residual_add/7: size:(?, 64, 64, 128) fn:add
  [TL] Conv2dLayer gen_B2A/res/c1/8: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins/8: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_B2A/res/c2/8: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/

  [TL] Conv2dLayer gen_B2A/res/c2/5: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins2/5: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_B2A/b_residual_add/5: size:(?, ?, ?, 128) fn:add
  [TL] Conv2dLayer gen_B2A/res/c1/6: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins/6: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_B2A/res/c2/6: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins2/6: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_B2A/b_residual_add/6: size:(?, ?, ?, 128) fn:add
  [TL] Conv2dLayer gen_B2A/res/c1/7: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins/7: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_B2A/res/c2/7: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A/res/ins2

In [10]:
## test inference
# gen_B_test, gen_B_test_logits = cyclegan_generator_resnet(real_A, 9, is_train=False, reuse=True, name='gen_A2B')
# gen_A_test, gen_A_test_logits = cyclegan_generator_resnet(real_B, 9, is_train=False, reuse=True, name='gen_B2A')

In [11]:
## calculate cycle loss
cyc_loss = tf.reduce_mean(tf.abs(cyc_A_out - real_A)) + tf.reduce_mean(tf.abs(cyc_B_out - real_B))

## calculate adversial loss
g_loss_A2B = tf.reduce_mean(tf.squared_difference(d_fake_B_logits, tf.ones_like(d_fake_B_logits)), name='g_loss_b')
g_loss_B2A = tf.reduce_mean(tf.squared_difference(d_fake_A_logits, tf.ones_like(d_fake_A_logits)),name='g_loss_a')        # g_loss_B2A = tf.reduce_mean(tf.reduce_mean(tf.squared_difference(d_fake_A, tf.ones_like(d_fake_A)), [1, 2, 3]), name='g_loss_a')

## calculate totalloss of generator
g_a2b_loss = lamda * cyc_loss + g_loss_A2B  # forward
g_b2a_loss = lamda * cyc_loss + g_loss_B2A  # backward

## calculate discriminator loss
d_a_loss = (tf.reduce_mean(tf.squared_difference(d_real_A_logits, tf.ones_like(d_real_A_logits))) 
                                                 + tf.reduce_mean(tf.square(d_fake_A_logits))) / 2.0
d_b_loss = (tf.reduce_mean(tf.squared_difference(d_real_B_logits, tf.ones_like(d_real_B_logits))) 
                                                 + tf.reduce_mean(tf.square(d_fake_B_logits))) / 2.0

In [12]:
# t_vars = tf.trainable_variables()
g_A2B_vars = tl.layers.get_variables_with_name('gen_A2B', True, True)
g_B2A_vars = tl.layers.get_variables_with_name('gen_B2A', True, True)
d_A_vars = tl.layers.get_variables_with_name('dis_A', True, True)
d_B_vars = tl.layers.get_variables_with_name('dis_B', True, True)

  [*] geting variables with gen_A2B
  got   0: gen_A2B/c7s1-32/W_conv2d:0   (7, 7, 3, 32)
  got   1: gen_A2B/c7s1-32/b_conv2d:0   (32,)
  got   2: gen_A2B/ins1/scale:0   (32,)
  got   3: gen_A2B/ins1/offset:0   (32,)
  got   4: gen_A2B/d64/W_conv2d:0   (3, 3, 32, 64)
  got   5: gen_A2B/d64/b_conv2d:0   (64,)
  got   6: gen_A2B/ins2/scale:0   (64,)
  got   7: gen_A2B/ins2/offset:0   (64,)
  got   8: gen_A2B/d128/W_conv2d:0   (3, 3, 64, 128)
  got   9: gen_A2B/d128/b_conv2d:0   (128,)
  got  10: gen_A2B/ins3/scale:0   (128,)
  got  11: gen_A2B/ins3/offset:0   (128,)
  got  12: gen_A2B/res/c1/0/W_conv2d:0   (3, 3, 128, 128)
  got  13: gen_A2B/res/c1/0/b_conv2d:0   (128,)
  got  14: gen_A2B/res/ins/0/scale:0   (128,)
  got  15: gen_A2B/res/ins/0/offset:0   (128,)
  got  16: gen_A2B/res/c2/0/W_conv2d:0   (3, 3, 128, 128)
  got  17: gen_A2B/res/c2/0/b_conv2d:0   (128,)
  got  18: gen_A2B/res/ins2/0/scale:0   (128,)
  got  19: gen_A2B/res/ins2/0/offset:0   (128,)
  got  20: gen_A2B/res/c1/1/W

In [13]:
# with tf.device('/gpu:0'):
with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(learning_rate, trainable=False)
    
g_a2b_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_a2b_loss, var_list=g_A2B_vars)
g_b2a_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_b2a_loss, var_list=g_B2A_vars)
d_a_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_a_loss, var_list=d_A_vars)
d_b_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_b_loss, var_list=d_B_vars)

In [15]:
## init params
tl.layers.initialize_global_variables(sess)

net_g_A2B_name = os.path.join(checkpoint_dir, '{}_net_g_A2B.npz'.format(dataset_dir))
net_g_B2A_name = os.path.join(checkpoint_dir, '{}_net_g_B2A.npz'.format(dataset_dir))
net_d_A_name = os.path.join(checkpoint_dir, '{}_net_d_A.npz'.format(dataset_dir))
net_d_B_name = os.path.join(checkpoint_dir, '{}_net_d_B.npz'.format(dataset_dir))

tl.files.load_and_assign_npz(sess=sess, name=net_g_A2B_name, network=gen_B)
tl.files.load_and_assign_npz(sess=sess, name=net_g_B2A_name, network=gen_A)
tl.files.load_and_assign_npz(sess=sess, name=net_d_A_name, network=d_fake_A)
tl.files.load_and_assign_npz(sess=sess, name=net_d_B_name, network=d_fake_B)

[!] Load checkpoint\sunflower2daisy_net_g_A2B.npz failed!
[!] Load checkpoint\sunflower2daisy_net_g_B2A.npz failed!
[!] Load checkpoint\sunflower2daisy_net_d_A.npz failed!
[!] Load checkpoint\sunflower2daisy_net_d_B.npz failed!


False

In [16]:
    ##========================= TRAIN MODELS ================================##
    iter_counter = 1
    start_time = time.time()

    dataA, dataB, im_test_A, im_test_B = tl.files.load_cyclegan_dataset(filename=dataset_dir, path='datasets')

    sample_A = np.asarray(dataA[0: 16])
    sample_B = np.asarray(dataB[0: 16])
    sample_A = tl.prepro.threading_data(sample_A, fn=rescale)
    sample_B = tl.prepro.threading_data(sample_B, fn=rescale)

    tl.vis.save_images(sample_A, [4, 4], './{}/sample_A.jpg'.format(sample_dir))
    tl.vis.save_images(sample_B, [4, 4], './{}/sample_B.jpg'.format(sample_dir))

    shuffle(dataA)
    shuffle(dataB)

    for epoch in range(epoch):
        ## change learning rate
        if epoch >= 100:
            new_lr = learning_rate - learning_rate * (epoch - 100) / 100
            sess.run(tf.assign(lr_v, new_lr))
            print("New learning rate %f" % new_lr)

        batch_idxs = min(min(len(dataA), len(dataB)), train_size) // batch_size
        for idx in range(0, batch_idxs):
            batch_imgA = tl.prepro.threading_data(dataA[idx * batch_size:(idx + 1) * batch_size], fn=prepro)
            batch_imgB = tl.prepro.threading_data(dataB[idx * batch_size:(idx + 1) * batch_size], fn=prepro)

            gen_A_temp_out, gen_B_temp_out = sess.run([gen_A_out, gen_B_out],
                                feed_dict={real_A: batch_imgA, real_B: batch_imgB})

            ## update forward network
            _, errGA2B = sess.run([g_a2b_optim, g_a2b_loss],
                                feed_dict={real_A: batch_imgA, real_B: batch_imgB})
            ## update DB network
            _, errDB = sess.run([d_b_optim, d_b_loss],
                                feed_dict={real_A: batch_imgA, real_B: batch_imgB, fake_B_pool: gen_B_temp_out})
            ## update (backword) network
            _, errGB2A = sess.run([g_b2a_optim, g_b2a_loss],
                                feed_dict={real_A: batch_imgA, real_B: batch_imgB})
            ## update DA network
            _, errDA = sess.run([d_a_optim, d_a_loss],
                                feed_dict={real_A: batch_imgA, real_B: batch_imgB, fake_A_pool: gen_A_temp_out})

            print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_a_loss: %.8f, d_b_loss: %.8f, g_a2b_loss: %.8f, g_b2a_loss: %.8f" \
                % (epoch, epoch, idx, batch_idxs, time.time() - start_time, errDA, errDB, errGA2B, errGB2A))

            iter_counter += 1
            # num_fake += 1

            if np.mod(iter_counter, 500) == 0:
                oA, oB = sess.run([gen_A_out, gen_B_out],
                                feed_dict={real_A: sample_A, real_B: sample_B})
                tl.vis.save_images(oA, [4, 4],
                                   './{}/B2A_{:02d}_{:04d}.jpg'.format(sample_dir, epoch, idx))
                print("save image gen_A, Epoch: %2d idx:%4d" % (epoch, idx))
                tl.vis.save_images(oB, [4, 4],
                                   './{}/A2B_{:02d}_{:04d}.jpg'.format(sample_dir, epoch, idx))
                print("save image gen_B, Epoch: %2d idx:%4d" % (epoch, idx))

        if (epoch != 0) and (epoch % 10) == 0:
            tl.files.save_npz(gen_B.all_params, name=net_g_A2B_name, sess=sess)
            tl.files.save_npz(gen_A.all_params, name=net_g_B2A_name, sess=sess)
            tl.files.save_npz(d_fake_A.all_params, name=net_d_A_name, sess=sess)
            tl.files.save_npz(d_fake_B.all_params, name=net_d_B_name, sess=sess)
            print("[*] Save checkpoints SUCCESS!")

Epoch: [ 0/ 0] [   0/  50] time: 44.5225s, d_a_loss: 2.61251187, d_b_loss: 1.25806665, g_a2b_loss: 14.26827049, g_b2a_loss: 16.44950867
Epoch: [ 0/ 0] [   1/  50] time: 85.8782s, d_a_loss: 1.34255135, d_b_loss: 1.67882121, g_a2b_loss: 14.75351429, g_b2a_loss: 15.90030956
Epoch: [ 0/ 0] [   2/  50] time: 127.2182s, d_a_loss: 1.68632078, d_b_loss: 2.50046754, g_a2b_loss: 14.38656425, g_b2a_loss: 14.48825836
Epoch: [ 0/ 0] [   3/  50] time: 168.4335s, d_a_loss: 1.43790352, d_b_loss: 1.85401762, g_a2b_loss: 14.82560349, g_b2a_loss: 14.26157379
Epoch: [ 0/ 0] [   4/  50] time: 209.7112s, d_a_loss: 1.46684086, d_b_loss: 1.15182543, g_a2b_loss: 12.53977871, g_b2a_loss: 12.57348156
Epoch: [ 0/ 0] [   5/  50] time: 251.0824s, d_a_loss: 1.38981342, d_b_loss: 1.70688689, g_a2b_loss: 12.99608040, g_b2a_loss: 12.11594486
Epoch: [ 0/ 0] [   6/  50] time: 292.2821s, d_a_loss: 1.52420950, d_b_loss: 1.96311903, g_a2b_loss: 13.39662361, g_b2a_loss: 13.17093754
Epoch: [ 0/ 0] [   7/  50] time: 333.4506s,

## Test cyclegan

In [17]:
def pro(x):
    x = x / (255. / 2.)
    x = x - 1.
    return x

In [19]:
test_A = tf.placeholder(tf.float32, [batch_size, image_size, image_size, c_dim], name='test_x')
test_B = tf.placeholder(tf.float32, [batch_size, image_size, image_size, c_dim], name='test_y')
test_gen_A2B, test_gen_A2B_logits = cyclegan_generator_resnet(test_A, 9, is_train=False, 
                                                              reuse=False, name='gen_A2B2')
test_gen_B2A, test_gen_B2A_logits = cyclegan_generator_resnet(test_B, 9, is_train=False, 
                                                              reuse=False, name='gen_B2A2')
    
out_var, in_var = (test_B, test_A) if direction == 'forward' else (test_A, test_B)

  [TL] InputLayer  gen_A2B2/in: (1, 256, 256, 3)
  [TL] Conv2dLayer gen_A2B2/c7s1-32: shape:[7, 7, 3, 32] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B2/ins1: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B2/d64: shape:[3, 3, 32, 64] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B2/ins2: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B2/d128: shape:[3, 3, 64, 128] strides:[1, 2, 2, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B2/ins3: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B2/res/c1/0: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B2/res/ins/0: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_A2B2/res/c2/0: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_A2B2/res/ins2/0: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_A2B2/b_residual_add/0: size:(1, 64, 64, 128) fn:add
  [TL] Conv2dLayer gen

  [TL] Conv2dLayer gen_B2A2/res/c2/6: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A2/res/ins2/6: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_B2A2/b_residual_add/6: size:(1, 64, 64, 128) fn:add
  [TL] Conv2dLayer gen_B2A2/res/c1/7: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A2/res/ins/7: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_B2A2/res/c2/7: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A2/res/ins2/7: epsilon:0.000010 act:identity
  [TL] ElementwiseLayer gen_B2A2/b_residual_add/7: size:(1, 64, 64, 128) fn:add
  [TL] Conv2dLayer gen_B2A2/res/c1/8: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer gen_B2A2/res/ins/8: epsilon:0.000010 act:relu
  [TL] Conv2dLayer gen_B2A2/res/c2/8: shape:[3, 3, 128, 128] strides:[1, 1, 1, 1] pad:SAME act:identity
  [TL] InstanceNormLayer g

In [20]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
tl.layers.initialize_global_variables(sess)

In [22]:
net_g_A2B_name = os.path.join(checkpoint_dir, '{}_net_g_A2B.npz'.format(dataset_dir))
net_g_B2A_name = os.path.join(checkpoint_dir, '{}_net_g_B2A.npz'.format(dataset_dir))
tl.files.load_and_assign_npz(sess=sess, name=net_g_A2B_name, network=test_gen_A2B)
tl.files.load_and_assign_npz(sess=sess, name=net_g_B2A_name, network=test_gen_B2A)

[!] Load checkpoint\sunflower2daisy_net_g_A2B.npz failed!
[!] Load checkpoint\sunflower2daisy_net_g_B2A.npz failed!


False

In [24]:
dataA, dataB, im_test_A, im_test_B = tl.files.load_cyclegan_dataset(filename=dataset_dir, path='datasets')

if direction == 'forward':
    sample_files = im_test_A
    net_g_logits = test_gen_A2B_logits

elif direction == 'backward':
    sample_files = im_test_B
    net_g_logits = test_gen_B2A_logits
else:
    raise Exception('--direction must be forward or backward')

In [25]:
batch_idxs = (len(sample_files)) // batch_size
    
for idx in range(0, batch_idxs):
    sample_image = threading_data(sample_files[idx * batch_size:(idx + 1) * batch_size],fn=pro)
    fake_img = sess.run(net_g_logits, feed_dict={in_var: sample_image})
    tl.vis.save_images(fake_img, [ni, ni], './{}/A2B_{}_{:04d}.jpg'.format(test_dir, direction, idx))

### Final Results:

<img src="assets/result.png" style="width: 600px;"/>

<img src="assets/result2.png" style="width: 600px;"/>